In [25]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import date, timedelta
import re

In [3]:
result_list = []
for num in range(90, 544):
    url = 'https://finance.naver.com/marketindex/interestDailyQuote.nhn?marketindexCd=IRR_CALL'
    params = {
        'page' : num
    }
    resp = requests.get(url, params = params)
    soup = BeautifulSoup(resp.content)
    tr_list = soup.find_all('tr')[1:]
    for tr in tr_list:
        time = tr.find_all('td')[0].text.strip()
        a = time.split('.')
        time2 = date(int(a[0]), int(a[1]), int(a[2]))
        price = tr.find_all('td')[1].text
        updown = tr.find_all('td')[3].text.rstrip("%").strip()
        
        new_dict = {
            'date' : time2,
            'price' : price,
            'upanddown' : updown
        }
        result_list.append(new_dict)

In [26]:
result_list

[{'date': datetime.date(2018, 1, 12), 'price': '1.50', 'upanddown': '0.00'},
 {'date': datetime.date(2018, 1, 11), 'price': '1.50', 'upanddown': '-0.66'},
 {'date': datetime.date(2018, 1, 10), 'price': '1.51', 'upanddown': '+ 1.34'},
 {'date': datetime.date(2018, 1, 9), 'price': '1.49', 'upanddown': '+ 2.05'},
 {'date': datetime.date(2018, 1, 8), 'price': '1.46', 'upanddown': '-0.68'},
 {'date': datetime.date(2018, 1, 5), 'price': '1.47', 'upanddown': '-0.68'},
 {'date': datetime.date(2018, 1, 4), 'price': '1.48', 'upanddown': '-1.33'},
 {'date': datetime.date(2018, 1, 3), 'price': '1.50', 'upanddown': '-1.32'},
 {'date': datetime.date(2018, 1, 2), 'price': '1.52', 'upanddown': '-3.80'},
 {'date': datetime.date(2017, 12, 29), 'price': '1.58', 'upanddown': '+ 1.94'},
 {'date': datetime.date(2017, 12, 28), 'price': '1.55', 'upanddown': '+ 2.65'},
 {'date': datetime.date(2017, 12, 27), 'price': '1.51', 'upanddown': '-0.66'},
 {'date': datetime.date(2017, 12, 26), 'price': '1.52', 'upanddo

In [27]:
result_df = pd.DataFrame(result_list)
result_df = result_df.set_index('date') # 이 과정이 없으면 merge가 안된다.
result_df.head()

,price,upanddown
date,,
2018-01-12,1.50,0.00
2018-01-11,1.50,-0.66
2018-01-10,1.51,+ 1.34
2018-01-09,1.49,+ 2.05
2018-01-08,1.46,-0.68


In [28]:
result_df

,price,upanddown
date,,
2018-01-12,1.50,0.00
2018-01-11,1.50,-0.66
2018-01-10,1.51,+ 1.34
2018-01-09,1.49,+ 2.05
2018-01-08,1.46,-0.68
...,...,...
2005-04-08,3.26,+ 0.93
2005-04-07,3.23,0.00
2005-04-06,3.23,-0.31


In [29]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3178 entries, 2018-01-12 to 2005-04-01
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   price      3178 non-null   object
 1   upanddown  3178 non-null   object
dtypes: object(2)
memory usage: 74.5+ KB


In [34]:
fulldate = pd.DataFrame(columns = ['date'])
start = date(2004, 8, 20)
end = date(2019, 7, 19)
for n in range(int((end - start).days) + 1):
    fulldate.loc[n, 'date'] = start + timedelta(n)
fulldate

,date
0,2004-08-20
1,2004-08-21
2,2004-08-22
3,2004-08-23
4,2004-08-24
...,...
5442,2019-07-15
5443,2019-07-16
5444,2019-07-17
5445,2019-07-18


In [58]:
rate = fulldate.merge(result_df, on = 'date', how = 'outer')
rate.head(10)

,date,price,upanddown
0,2004-08-20,NaN,NaN
1,2004-08-21,NaN,NaN
2,2004-08-22,NaN,NaN
3,2004-08-23,NaN,NaN
4,2004-08-24,NaN,NaN
5,2004-08-25,NaN,NaN
6,2004-08-26,NaN,NaN
7,2004-08-27,NaN,NaN
8,2004-08-28,NaN,NaN
9,2004-08-29,NaN,NaN


In [70]:
rate['price'] = rate['price'].fillna(method='ffill')
rate['upanddown'] = rate['upanddown'].fillna(method='bfill')

In [71]:
rate.head()

,date,price,upanddown
0,2004-08-20,3.25,-0.31
1,2004-08-21,3.25,-0.31
2,2004-08-22,3.25,-0.31
3,2004-08-23,3.25,-0.31
4,2004-08-24,3.25,-0.31


In [72]:
rate = rate.set_index('date')

In [73]:
rate.head()

,price,upanddown
date,,
2004-08-20,3.25,-0.31
2004-08-21,3.25,-0.31
2004-08-22,3.25,-0.31
2004-08-23,3.25,-0.31
2004-08-24,3.25,-0.31


In [32]:
# rate['price']=rate['price'].fillna(method='pad')
# rate['upanddown']=rate['upanddown'].fillna(method='pad')
# rate.to_csv("callrate_new.csv")

In [48]:
# rate[224:]

,date,price,upanddown
224,2005-04-01,3.25,-0.31
225,2005-04-02,3.25,-0.31
226,2005-04-03,3.25,-0.31
227,2005-04-04,3.24,-0.31
228,2005-04-05,3.24,-0.31
...,...,...,...
5442,2019-07-15,1.50,0.00
5443,2019-07-16,1.50,0.00
5444,2019-07-17,1.50,0.00
5445,2019-07-18,1.50,0.00


In [74]:
def up_down_call(df):
    df_ud = pd.DataFrame(index = df.index, columns = ['up_down'])
    for i in range(len(df.index)):
        time = df.index[i]
        time2 = df.index[i] - pd.DateOffset(months = 1)
        
        if time.year >= 2005:

            while time2.date() not in df.index:
                time2 = time2 - pd.DateOffset(days = 1)
                
            if time2.date() in df.index:

                if df.loc[time].price > df.loc[time2.date()].price:
                    df_ud['up_down'][i] = '상승'
                elif df.loc[time].price == df.loc[time2.date()].price:
                    df_ud['up_down'][i] = 0
                else:
                    df_ud['up_down'][i] = '하락'
    return df_ud

In [76]:
M = up_down_call(rate)
M[250:]

,up_down
date,
2005-04-27,상승
2005-04-28,상승
2005-04-29,상승
2005-04-30,상승
2005-05-01,상승
...,...
2019-07-15,0
2019-07-16,0
2019-07-17,0


In [78]:
df_a = rate.join(M, on = 'date', how = 'outer')
df_call = df_a.loc[date(2005, 4, 1):]

In [79]:
df_call

,price,upanddown,up_down
date,,,
2005-04-01,3.25,-0.31,0
2005-04-02,3.25,-0.31,0
2005-04-03,3.25,-0.31,0
2005-04-04,3.24,-0.31,하락
2005-04-05,3.24,-0.31,하락
...,...,...,...
2019-07-15,1.50,0.00,0
2019-07-16,1.50,0.00,0
2019-07-17,1.50,0.00,0


In [107]:
df_call.to_csv("callrate_new.csv")

In [118]:
day = date(2019, 7, 1)
a = day - pd.DateOffset(months = 1)
a

Timestamp('2019-06-01 00:00:00')

In [119]:
day = date(2005, 5, 1)
b = day - pd.DateOffset(months = 1)
print(b)
print(b.date())
df_call.loc[b.date()].price

2005-04-01 00:00:00
2005-04-01


'3.25'

In [120]:
def up_down_call(df):
    df_ud = pd.DataFrame(index = df.index, columns = ['up_down'])
    for i in range(len(df.index)):
        time = df.index[i]
        time2 = df.index[i] - pd.DateOffset(months = 1)
        
        if time.year >= 2005:

            while time2.date() not in df.index:
                time2 = time2 - pd.DateOffset(days = 1)
                
            if time2.date() in df.index:

                if df.loc[time].price > df.loc[time2.date()].price:
                    df_ud['up_down'][i] = '상승'
                elif df.loc[time].price == df.loc[time2.date()].price:
                    df_ud['up_down'][i] = 0
                else:
                    df_ud['up_down'][i] = '하락'
    return df_ud

In [121]:
df_t = up_down_call(df_call)
df_t.head()

OutOfBoundsDatetime: ignored

In [102]:
df_a = M.join(df_t, on = 'date', how = 'outer')
df_call = df_a.loc[:date(2005, 4, 1)]
df_call

ValueError: ignored